# I will be testing out different assumption and tools

In [20]:
import librosa
import pywt
# from pyAudioAnalysis import audioBasicIO # doesnt support mp3
# from pyAudioAnalysis import ShortTermFeatures
import numpy as np

In [21]:
# get sample rate of audio file in .wav and mp3. Parameter: file path
wav_sr = librosa.core.get_samplerate('/Users/macbookretina/blues.00042.wav')
mp3_sr = librosa.core.get_samplerate('/Users/macbookretina/073192.mp3')
wav_sr, mp3_sr

(22050, 44100)

In [22]:
# load audio file in .wav and mp3 format. Parameters:
# path: (string, int, pathlib.Path or file-like object) path to the input file.
# sr: int - target sampling rate
# mono:bool - convert signal to mono
# offset:float - start reading after this time (in seconds)
# duration:float - only load up to this much audio (in seconds)
# dtype:numeric type - data type of y
# res_type:str - resample type 
y, sr = librosa.core.load('/Users/macbookretina/073192.mp3', mp3_sr)
y, sr = librosa.core.load('/Users/macbookretina/073192.mp3', mp3_sr)
sr, y

/Users/macbookretina/Library/Python/3.7/lib/python/site-packages/librosa/core/audio.py:161: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn('PySoundFile failed. Trying audioread instead.')
/Users/macbookretina/Library/Python/3.7/lib/python/site-packages/librosa/core/audio.py:161: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn('PySoundFile failed. Trying audioread instead.')


(44100, array([0., 0., 0., ..., 0., 0., 0.], dtype=float32))

In [98]:
# get duration of audio in seconds
duration = librosa.core.get_duration(y, mp3_sr)
duration

30.01469387755102

# Timbral Features

In [176]:
# Computer Linear Prediction Coefficients via Burg’s method. Parameters:
# y:np.ndarray - Time series to fit
# order:int > 0 - Order of the linear filter
# Question: How to determine which order to use
# 
# Returns: a : np.ndarray of length order + 1 - LP prediction error coefficients, i.e. filter denominator polynomial
lpc = librosa.lpc(y, 3)
lpc

array([ 1.        , -1.9204183 ,  1.2394348 , -0.29715297], dtype=float32)

In [60]:
# Computer Spectral Centroid. Parameters:
# y:np.ndarray [shape=(n,)] or None - audio time series
# sr:number > 0 [scalar] - audio sampling rate of y
# S:np.ndarray [shape=(d, t)] or None - (optional) spectrogram magnitude
# n_fft:int > 0 [scalar] - FFT window size
# hop_length:int > 0 [scalar] - hop length for STFT. See librosa.core.stft for details.
# freq:None or np.ndarray [shape=(d,) or shape=(d, t)] - Center frequencies for spectrogram bins. If None, then FFT bin center frequencies are used. Otherwise, it can be a single array of d center frequencies, or a matrix of center frequencies as constructed by librosa.core.ifgram
# win_length:int <= n_fft [scalar] - Each frame of audio is windowed by window(). The window will be of length win_length and then padded with zeros to match n_fft.

# Returns: centroid : np.ndarray [shape=(1, t)] - centroid frequencies
spec_centroid = librosa.feature.spectral_centroid(y, mp3_sr)
mean_spec_centroid = np.mean(spec_centroid)
mean_spec_centroid
# 1233.136154764339

2466.272309528678

In [23]:
# Computer Spectral Roll-off. Parameters:
# y:np.ndarray [shape=(n,)] or None - audio time series
# sr:number > 0 [scalar] - audio sampling rate of y
# S:np.ndarray [shape=(d, t)] or None - (optional) spectrogram magnitude
# n_fft:int > 0 [scalar] - FFT window size
# hop_length:int > 0 [scalar] - hop length for STFT. See librosa.core.stft for details.
# freq:None or np.ndarray [shape=(d,) or shape=(d, t)] - Center frequencies for spectrogram bins. If None, then FFT bin center frequencies are used. Otherwise, it can be a single array of d center frequencies, or a matrix of center frequencies as constructed by librosa.core.ifgram
# win_length:int <= n_fft [scalar] - Each frame of audio is windowed by window(). The window will be of length win_length and then padded with zeros to match n_fft.

# Returns: rolloff : np.ndarray [shape=(1, t)] - roll-off frequency for each frame
spec_rolloff = librosa.feature.spectral_rolloff(y, mp3_sr)
mean_spec_rolloff = np.mean(spec_rolloff)
spec_rolloff.shape

(1, 2586)

In [24]:
# Compute zero crossing rate. Parameters:
# y:np.ndarray [shape=(n,)] - Audio time series
# frame_length:int > 0 - Length of the frame over which to compute zero crossing rates
# hop_length:int > 0 - Number of samples to advance for each frame
# center:bool - If True, frames are centered by padding the edges of y. This is similar to the padding in librosa.core.stft, but uses edge-value copies instead of reflection.
# kwargs:additional keyword arguments
# See librosa.core.zero_crossings

# Returns: zcr : np.ndarray [shape=(1, t)] - `zcr[0, i]` is the fraction of zero crossings in the `i` th frame
zcr = librosa.feature.zero_crossing_rate(y)
mean_zcr = np.mean(zcr)
zcr.shape

(1, 2586)

In [63]:
# Compute the first 13 mfcc coefficients. Parameters:
# y:np.ndarray [shape=(n,)] or None - audio time series
# sr:number > 0 [scalar] - sampling rate of y
# S:np.ndarray [shape=(d, t)] or None - log-power Mel spectrogram
# n_mfcc: int > 0 [scalar] - number of MFCCs to return
# dct_type:{1, 2, 3} - Discrete cosine transform (DCT) type. By default, DCT type-2 is used.
# norm:None or ‘ortho’ - If dct_type is 2 or 3, setting norm=’ortho’ uses an ortho-normal DCT basis. Normalization is not supported for dct_type=1.
# lifter:number >= 0 - If lifter>0, apply liftering (cepstral filtering) to the MFCCs:
# M[n, :] <- M[n, :] * (1 + sin(pi * (n + 1) / lifter)) * lifter / 2
# Setting lifter >= 2 * n_mfcc emphasizes the higher-order coefficients. As lifter increases, the coefficient weighting becomes approximately linear.
# kwargs:additional keyword arguments
# Arguments to melspectrogram, if operating on time series input
# hop_length:int > 0 - The number of samples to advance between frames.
# mfcc = librosa.feature.mfcc(y=y, sr=sr, hop_length=512, n_mfcc=13)

# Returns: M : np.ndarray [shape=(n_mfcc, t)] - MFCC sequence
mfcc = librosa.feature.mfcc(y=y, sr=mp3_sr, n_mfcc=13)
mfcc_scaled = np.mean(mfcc.T, axis=0)
mfcc_scaled

array([-126.35341  ,  168.79324  ,  -51.67301  ,   31.850351 ,
         -7.024031 ,   24.488293 ,  -23.559675 ,   10.718372 ,
        -11.275455 ,    5.5729837,   -8.264879 ,    5.572796 ,
         -5.336298 ], dtype=float32)

In [197]:
# Computer log-mel / constant-Q transform of an audio signal. Parameters:
# y:np.ndarray [shape=(n,)] - audio time series
# sr:number > 0 [scalar] - sampling rate of y
# hop_length:int > 0 [scalar] - number of samples between successive CQT columns.
# fmin:float > 0 [scalar] - Minimum frequency. Defaults to C1 ~= 32.70 Hz
# n_bins:int > 0 [scalar] - Number of frequency bins, starting at fmin
# n_bins = 84 by default

# Returns: CQT : np.ndarray [shape=(n_bins, t), dtype=np.complex or np.float] - Constant-Q value each frequency at each time.
cqt = librosa.cqt(y, sr=mp3_sr)
# abs_cqt = np.abs(cqt)
# scaled_abs_cqt = np.mean(abs_cqt.T, axis=0)
# scaled_abs_cqt
cqt.shape, cqt

((84, 2586),
 array([[ 4.04311841e-01+8.32610300e-04j, -2.98204379e-01+2.70791944e-01j,
          3.77148955e-02-3.97318771e-01j, ...,
         -5.43648534e-02+2.00999616e-01j, -1.19690511e-01-1.55654746e-01j,
          1.92880422e-01-4.46009962e-03j],
        [ 4.08876415e-01-1.06428507e-03j, -3.25997050e-01+2.41493339e-01j,
          1.16567770e-01-3.80030661e-01j, ...,
         -9.85589980e-02-1.57296378e-01j,  1.53900200e-01+8.53853488e-02j,
         -1.73288048e-01-2.70970694e-03j],
        [ 2.83232916e-01-1.87611900e-03j, -2.44327878e-01+1.40942486e-01j,
          1.41804350e-01-2.40311015e-01j, ...,
          2.23164502e-02-9.00796783e-02j,  3.99101756e-02+5.80575931e-02j,
         -6.24006198e-02+4.88761364e-03j],
        ...,
        [ 0.00000000e+00+0.00000000e+00j, -1.04026048e-07-2.36192019e-07j,
         -2.03580000e-02+8.53440893e-03j, ...,
         -2.34571101e-02-1.16492463e-02j, -2.13700193e-02-9.03730050e-02j,
          0.00000000e+00+0.00000000e+00j],
        [ 0.00

In [196]:
# Computer Mel-spectogram. Parameters:
# y:np.ndarray [shape=(n,)] or None - audio time-series
# sr:number > 0 [scalar] - sampling rate of y
# S:np.ndarray [shape=(d, t)] - spectrogram
# n_fft:int > 0 [scalar] - length of the FFT window
# hop_length:int > 0 [scalar] - number of samples between successive frames. See librosa.core.stft
# win_length:int <= n_fft [scalar] - Each frame of audio is windowed by window(). The window will be of length win_length and then padded with zeros to match n_fft.
# If unspecified, defaults to win_length = n_fft.
# window:string, tuple, number, function, or np.ndarray [shape=(n_fft,)]
# - a window specification (string, tuple, or number); see scipy.signal.get_window
# - a window function, such as scipy.signal.hanning
# - a vector or array of length n_fft
# center:boolean
# - If True, the signal y is padded so that frame t is centered at y[t * hop_length].
# - If False, then frame t begins at y[t * hop_length]
# pad_mode:string - If center=True, the padding mode to use at the edges of the signal. By default, STFT uses reflection padding.
# power:float > 0 [scalar] - Exponent for the magnitude melspectrogram. e.g., 1 for energy, 2 for power, etc.
# kwargs:additional keyword arguments
# Mel filter bank parameters. See librosa.filters.mel for details.

# Returns: S : np.ndarray [shape=(n_mels, t)] - Mel spectrogram
mel_spect = librosa.feature.melspectrogram(y=y, sr=mp3_sr)
# mel_spect_p = librosa.power_to_db(mel_spect, ref=np.max) # convert spectogram to decibels unit
mel_spect.shape, mel_spect

((128, 2586),
 array([[1.8680854e-05, 6.2002707e-02, 1.1151104e+00, ..., 1.7733978e-01,
         2.8754392e-01, 1.6142677e-01],
        [6.5987210e-06, 2.5891724e-01, 2.1808436e+00, ..., 8.4348541e-01,
         4.4655958e-01, 2.6228645e-01],
        [3.2318534e-05, 4.6489990e-01, 2.0627596e+00, ..., 3.9542034e+00,
         3.7528203e+00, 5.1367295e-01],
        ...,
        [1.1018846e-11, 3.4859121e-10, 1.1011653e-09, ..., 5.1674506e-06,
         4.3752367e-05, 2.4380066e-05],
        [2.7074251e-11, 6.7583716e-10, 1.4731847e-09, ..., 5.1306511e-06,
         4.3441189e-05, 2.4208157e-05],
        [1.3928958e-11, 3.4815550e-10, 1.1026061e-09, ..., 5.1061379e-06,
         4.3238626e-05, 2.4084664e-05]], dtype=float32))

In [198]:
# Compute spectral bandwidth
# Parameters:	
# y:np.ndarray [shape=(n,)] or None - audio time series
# sr:number > 0 [scalar] - audio sampling rate of y
# S:np.ndarray [shape=(d, t)] or None - (optional) spectrogram magnitude
# n_fft:int > 0 [scalar] - FFT window size
# hop_length:int > 0 [scalar] - hop length for STFT. See librosa.core.stft for details.
# win_length:int <= n_fft [scalar] - Each frame of audio is windowed by window(). The window will be of length
#     win_length and then padded with zeros to match n_fft. If unspecified, defaults to win_length = n_fft.

# Returns: bandwidth:np.ndarray [shape=(1, t)] - frequency bandwidth for each frame
spec_bw = librosa.feature.spectral_bandwidth(y=y, sr=mp3_sr)
spec_bw.shape, spec_bw

((1, 2586),
 array([[2235.57868228, 2463.75327568, 2530.21258602, ..., 3400.60647967,
         3574.43060148, 3803.22950521]]))

In [199]:
# Compute spectral contrast. Parameters:
# y:np.ndarray [shape=(n,)] or None - audio time series
# sr:number > 0 [scalar] - audio sampling rate of y
# S:np.ndarray [shape=(d, t)] or None - (optional) spectrogram magnitude
# n_fft:int > 0 [scalar] - FFT window size
# hop_length:int > 0 [scalar] - hop length for STFT. See librosa.core.stft for details.
# win_length:int <= n_fft [scalar] - Each frame of audio is windowed by window(). The window will be of length win_length and then padded with zeros to match n_fft.
# If unspecified, defaults to win_length = n_fft.
# window:string, tuple, number, function, or np.ndarray [shape=(n_fft,)]
# - a window specification (string, tuple, or number); see scipy.signal.get_window
# - a window function, such as scipy.signal.hanning
# - a vector or array of length n_fft
# center:boolean
# - If True, the signal y is padded so that frame t is centered at y[t * hop_length].
# - If False, then frame t begins at y[t * hop_length]
# pad_mode:string - If center=True, the padding mode to use at the edges of the signal. By default, STFT uses reflection padding.
# freq:None or np.ndarray [shape=(d,)] - Center frequencies for spectrogram bins. If None, then FFT bin center frequencies are used. Otherwise, it can be a single array of d center frequencies.
# fmin:float > 0 - Frequency cutoff for the first bin [0, fmin] Subsequent bins will cover [fmin, 2*fmin], [2*fmin, 4*fmin], etc.
# n_bands:int > 1 - number of frequency bands
# quantile:float in (0, 1) - quantile for determining peaks and valleys
# linear:bool
# If True, return the linear difference of magnitudes: peaks - valleys.
# If False, return the logarithmic difference: log(peaks) - log(valleys).

# Returns: contrast:np.ndarray [shape=(n_bands + 1, t)] - each row of spectral contrast values corresponds to a given octave-based frequency
contrast = librosa.feature.spectral_contrast(y=y, sr=mp3_sr)
contrast.shape, contrast

((7, 2586), array([[ 8.04118339, 10.38297466,  8.48753499, ..., 19.02623835,
          9.25938174,  9.34614714],
        [15.53301358,  8.55531761,  8.01460751, ...,  4.85495519,
          4.89166447,  3.94521694],
        [27.39611174, 17.39314195,  7.30296729, ..., 10.19008337,
         10.00324236, 10.63376905],
        ...,
        [20.95746158, 13.56260854, 17.60133196, ..., 15.80464073,
         12.37423322, 10.47144242],
        [31.08062994, 17.99741829, 18.65663564, ..., 14.77623737,
         11.03944072, 12.43721879],
        [33.63363901, 41.82228868, 44.47098729, ..., 22.06003762,
         18.08813225, 15.68319987]]))

In [18]:
# Compute spectral flux
[Fs, x] = audioBasicIO.read_audio_file("/Users/macbookretina/blues.00042.wav")
F, f_names = ShortTermFeatures.feature_extraction(x, Fs, 0.050*Fs, 0.025*Fs)
# F, f_names
# len(F) == len(f_names)
F[6]

array([0.        , 0.00405326, 0.00223896, ..., 0.00088491, 0.00074295,
       0.00077725])

# Rhythmic Features

In [66]:
# Compute tempo and beat. Parameters:
# y:np.ndarray [shape=(n,)] or None - audio time series
# sr:number > 0 [scalar] - sampling rate of y
# onset_envelope:np.ndarray [shape=(n,)] or None - (optional) pre-computed onset strength envelope.
# hop_length:int > 0 [scalar] - number of audio samples between successive onset_envelope values
# start_bpm:float > 0 [scalar] - initial guess for the tempo estimator (in beats per minute)
# tightness:float [scalar] - tightness of beat distribution around tempo
# trim:bool [scalar] - trim leading/trailing beats with weak onsets
# bpm:float [scalar] - (optional) If provided, use bpm as the tempo instead of estimating it from onsets.
# prior:scipy.stats.rv_continuous [optional] - An optional prior distribution over tempo. If provided, start_bpm will be ignored.
# units:{‘frames’, ‘samples’, ‘time’} - The units to encode detected beat events in. By default, ‘frames’ are used.

# returns the estimated global tempo (beats/s) and estimated beat event locations in the specified units.
tempo, beats = librosa.beat.beat_track(y=y, sr=mp3_sr)
tempo

147.65625

In [183]:
beats.shape, beats

((71,),
 array([   3,   36,   71,  107,  143,  179,  216,  251,  288,  323,  361,
         397,  435,  470,  506,  544,  581,  617,  655,  691,  729,  765,
         800,  830,  862,  894,  929,  964, 1000, 1037, 1072, 1109, 1144,
        1180, 1214, 1250, 1284, 1320, 1355, 1389, 1425, 1459, 1495, 1531,
        1567, 1602, 1636, 1671, 1705, 1741, 1776, 1810, 1846, 1880, 1914,
        1948, 1986, 2020, 2055, 2090, 2126, 2161, 2196, 2232, 2267, 2302,
        2336, 2371, 2405, 2440, 2475]))

In [184]:
# Convert beats to timestamps. Parameters:
# frames: np.ndarray [shape=(n,)] - frame index or vector of frame indices
# sr: number > 0 [scalar] - audio sampling rate
# hop_length : int > 0 [scalar] - number of samples between successive frames
# n_fft : None or int > 0 [scalar] - Optional: length of the FFT window.
# If given, time conversion will include an offset of `n_fft / 2` to counteract windowing effects when using a non-centered STFT.

# Returns: times : np.ndarray [shape=(n,)] - time (in seconds) of each given frame number: `times[i] = frames[i] * hop_length / sr`
beat_timestamps = librosa.frames_to_time(beats, sr=mp3_sr)
beat_timestamps.shape, beat_timestamps

((71,), array([ 0.03482993,  0.41795918,  0.82430839,  1.24226757,  1.66022676,
         2.07818594,  2.5077551 ,  2.91410431,  3.34367347,  3.75002268,
         4.19120181,  4.609161  ,  5.05034014,  5.45668934,  5.87464853,
         6.31582766,  6.74539683,  7.16335601,  7.60453515,  8.02249433,
         8.46367347,  8.88163265,  9.28798186,  9.63628118, 10.00780045,
        10.37931973, 10.78566893, 11.19201814, 11.60997732, 12.03954649,
        12.44589569, 12.87546485, 13.28181406, 13.69977324, 14.09451247,
        14.51247166, 14.90721088, 15.32517007, 15.73151927, 16.1262585 ,
        16.54421769, 16.93895692, 17.3569161 , 17.77487528, 18.19283447,
        18.59918367, 18.9939229 , 19.40027211, 19.79501134, 20.21297052,
        20.61931973, 21.01405896, 21.43201814, 21.82675737, 22.2214966 ,
        22.61623583, 23.05741497, 23.4521542 , 23.8585034 , 24.26485261,
        24.68281179, 25.089161  , 25.4955102 , 25.91346939, 26.31981859,
        26.7261678 , 27.12090703, 27.5272562

In [200]:
# Compute predominant local pulse (PLP) estimation.
onset_env = librosa.onset.onset_strength(y=y, sr=mp3_sr)
pulse = librosa.beat.plp(onset_envelope=onset_env, sr=mp3_sr)
pulse.shape, pulse

((2586,),
 array([1.        , 0.98977244, 0.97675943, ..., 0.01638729, 0.        ,
        0.        ], dtype=float32))

In [185]:
# Compute the tempogram: local autocorrelation of the onset strength envelope. Parameters:
# y : np.ndarray [shape=(n,)] or None - Audio time series.
# sr : number > 0 [scalar] - sampling rate of `y`
# onset_envelope : np.ndarray [shape=(n,) or (m, n)] or None - Optional pre-computed onset strength envelope as provided by `onset.onset_strength`.
# If multi-dimensional, tempograms are computed independently for each band (first dimension).
# hop_length : int > 0 - number of audio samples between successive onset measurements
# win_length : int > 0 - length of the onset autocorrelation window (in frames/onset measurements)
# The default settings (384) corresponds to `384 * hop_length / sr ~= 8.9s`.
# center : bool - If `True`, onset autocorrelation windows are centered. If `False`, windows are left-aligned.
# window : string, function, number, tuple, or np.ndarray [shape=(win_length,)] - A window specification as in `core.stft`.
# norm : {np.inf, -np.inf, 0, float > 0, None} - Normalization mode.  Set to `None` to disable normalization.

# Returns: tempogram : np.ndarray [shape=(win_length, n) or (m, win_length, n)] - Localized autocorrelation of the onset strength envelope.
# If given multi-band input (`onset_envelope.shape==(m,n)`) then `tempogram[i]` is the tempogram of `onset_envelope[i]`.
hop_length = 512
oenv = librosa.onset.onset_strength(y=y, sr=mp3_sr, hop_length=hop_length)
tempogram = librosa.feature.tempogram(onset_envelope=oenv, sr=mp3_sr, hop_length=hop_length)
# scaled_tempogram = np.mean(tempogram.T, axis=0)
tempogram.shape, tempogram

((384, 2586), array([[ 1.00000000e+00,  1.00000000e+00,  1.00000000e+00, ...,
          1.00000000e+00,  1.00000000e+00,  1.00000000e+00],
        [ 4.57723276e-01,  4.58392928e-01,  4.59072800e-01, ...,
          9.62856608e-01,  9.62974336e-01,  9.63093458e-01],
        [ 2.13487319e-01,  2.14414398e-01,  2.15356259e-01, ...,
          9.37953159e-01,  9.38207345e-01,  9.38464002e-01],
        ...,
        [-2.07094156e-17,  4.17646944e-17,  8.36791841e-17, ...,
          3.44379609e-12,  2.16006357e-12,  1.17101767e-12],
        [-4.79612356e-19,  2.94727784e-17,  8.43408088e-17, ...,
          3.84040333e-13,  2.33028881e-13,  9.48426750e-14],
        [ 1.22055196e-18,  1.11708335e-16,  9.78448950e-17, ...,
         -6.32146292e-16, -1.57938070e-15, -4.59024353e-16]]))

In [186]:
# Compute global onset autocorrelation
ac_global = librosa.autocorrelate(oenv, max_size=tempogram.shape[0])
ac_global = librosa.util.normalize(ac_global)
mean_ac_global = np.mean(ac_global)
ac_global.shape, ac_global

((384,), array([1.        , 0.81841422, 0.71248654, 0.66720819, 0.6465774 ,
        0.64140223, 0.64063488, 0.6417909 , 0.64056895, 0.64118979,
        0.64204841, 0.64340545, 0.64623867, 0.65289392, 0.6611401 ,
        0.67255562, 0.69392766, 0.71669094, 0.73067842, 0.71035846,
        0.68301238, 0.66320777, 0.65199921, 0.64461936, 0.64274344,
        0.64403062, 0.64279767, 0.64140545, 0.63964226, 0.63626039,
        0.63479852, 0.63839811, 0.652379  , 0.67659723, 0.70832027,
        0.73375964, 0.72515291, 0.69785915, 0.67773522, 0.65599069,
        0.64552468, 0.63794335, 0.63455404, 0.63310334, 0.63482194,
        0.63686132, 0.63722284, 0.64100112, 0.6380514 , 0.64204197,
        0.6515414 , 0.67189653, 0.69003341, 0.70157373, 0.69666234,
        0.68673904, 0.67133073, 0.65788617, 0.65006026, 0.64088437,
        0.63508501, 0.63002481, 0.62899499, 0.62434256, 0.6224777 ,
        0.62102502, 0.62666737, 0.63931911, 0.65907819, 0.68801103,
        0.70436992, 0.69949916, 0.687007

In [190]:
# Compute the Fourier tempogram: the short-time Fourier transform of the onset strength envelope. Parameters:
# y : np.ndarray [shape=(n,)] or None - Audio time series.
# sr : number > 0 [scalar] - sampling rate of `y`
# onset_envelope : np.ndarray [shape=(n,)] or None - Optional pre-computed onset strength envelope as provided by `onset.onset_strength`.
# hop_length : int > 0 - number of audio samples between successive onset measurements
# win_length : int > 0 - length of the onset window (in frames/onset measurements) The default settings (384) corresponds to `384 * hop_length / sr ~= 8.9s`.
# center : bool - If `True`, onset windows are centered. If `False`, windows are left-aligned.
# window : string, function, number, tuple, or np.ndarray [shape=(win_length,)] - A window specification as in `core.stft`.

# Returns: tempogram : np.ndarray [shape=(win_length // 2 + 1, n)] - Complex short-time Fourier transform of the onset envelope.
fourier_tempogram = librosa.feature.fourier_tempogram(onset_envelope=oenv, sr=mp3_sr, hop_length=hop_length)
abs_fourier_tempogram = np.abs(fourier_tempogram)
fourier_tempogram.shape, fourier_tempogram

((193, 2587),
 array([[ 3.0548050e+02+0.00000000e+00j,  3.0547522e+02+0.00000000e+00j,
          3.0545944e+02+0.00000000e+00j, ...,
          2.3386009e+02+0.00000000e+00j,  2.3386099e+02+0.00000000e+00j,
          2.3386009e+02+0.00000000e+00j],
        [-1.9258073e+02+2.53249705e-14j, -1.9256468e+02-1.30363035e+00j,
         -1.9251648e+02-2.60638213e+00j, ...,
         -1.2047324e+02+1.15963086e-01j, -1.2047425e+02+2.32720540e-14j,
         -1.2047324e+02-1.15963086e-01j],
        [ 8.1986847e+01-1.50662902e-16j,  8.1936127e+01+2.72005963e+00j,
          8.1784126e+01+5.43602085e+00j, ...,
          4.7884474e+00-1.19152024e-01j,  4.7903652e+00+2.67940937e-15j,
          4.7884474e+00+1.19152024e-01j],
        ...,
        [-4.2847584e+01+4.59155512e-15j,  4.2822857e+01-1.36669731e+00j,
         -4.2748791e+01+2.73144174e+00j, ...,
          5.1401973e+00+1.93231821e-01j, -5.1442046e+00-1.45816415e-15j,
          5.1401973e+00-1.93231821e-01j],
        [ 4.3673531e+01+1.76447180e-1

In [191]:
abs_fourier_tempogram.shape, abs_fourier_tempogram

((193, 2587),
 array([[3.0548050e+02, 3.0547522e+02, 3.0545944e+02, ..., 2.3386009e+02,
         2.3386099e+02, 2.3386009e+02],
        [1.9258073e+02, 1.9256909e+02, 1.9253412e+02, ..., 1.2047330e+02,
         1.2047425e+02, 1.2047330e+02],
        [8.1986847e+01, 8.1981262e+01, 8.1964584e+01, ..., 4.7899294e+00,
         4.7903652e+00, 4.7899294e+00],
        ...,
        [4.2847584e+01, 4.2844662e+01, 4.2835964e+01, ..., 5.1438279e+00,
         5.1442046e+00, 5.1438279e+00],
        [4.3673531e+01, 4.3670914e+01, 4.3662994e+01, ..., 1.9101042e+00,
         1.9099224e+00, 1.9101042e+00],
        [4.2350616e+01, 4.2347393e+01, 4.2337791e+01, ..., 2.0005128e-01,
         2.0028403e-01, 2.0005128e-01]], dtype=float32))

In [192]:
# Compute the auto-correlation tempogram, unnormalized to make comparison easier
ac_fourier_tempogram = librosa.feature.tempogram(onset_envelope=oenv, sr=sr, hop_length=hop_length, norm=None)
mean_ac_fourier_tempogram = np.mean(ac_fourier_tempogram)
ac_fourier_tempogram.shape, ac_fourier_tempogram

((384, 2586), array([[ 1.12701414e+03,  1.12924063e+03,  1.13119426e+03, ...,
          1.38927932e+02,  1.37469666e+02,  1.36006229e+02],
        [ 5.15860606e+02,  5.17635916e+02,  5.19300517e+02, ...,
          1.33767678e+02,  1.32379760e+02,  1.30986710e+02],
        [ 2.40603228e+02,  2.42125449e+02,  2.43609765e+02, ...,
          1.30307893e+02,  1.28975050e+02,  1.27636950e+02],
        ...,
        [-2.33398043e-14,  4.71623896e-14,  9.46574130e-14, ...,
          4.78439470e-10,  2.96943216e-10,  1.59265698e-10],
        [-5.40529910e-16,  3.32818587e-14,  9.54058390e-14, ...,
          5.33539295e-11,  3.20344024e-11,  1.28991946e-11],
        [ 1.37557932e-15,  1.26145590e-13,  1.10681584e-13, ...,
         -8.78227773e-14, -2.17116937e-13, -6.24301714e-14]]))

# Wavelet

In [99]:
# Buit-in wavelet faimilies (mother wavelets)
pywt.families()

['haar',
 'db',
 'sym',
 'coif',
 'bior',
 'rbio',
 'dmey',
 'gaus',
 'mexh',
 'morl',
 'cgau',
 'shan',
 'fbsp',
 'cmor']

In [100]:
# Builtin specific wavelet
pywt.wavelist()

['bior1.1',
 'bior1.3',
 'bior1.5',
 'bior2.2',
 'bior2.4',
 'bior2.6',
 'bior2.8',
 'bior3.1',
 'bior3.3',
 'bior3.5',
 'bior3.7',
 'bior3.9',
 'bior4.4',
 'bior5.5',
 'bior6.8',
 'cgau1',
 'cgau2',
 'cgau3',
 'cgau4',
 'cgau5',
 'cgau6',
 'cgau7',
 'cgau8',
 'cmor',
 'coif1',
 'coif2',
 'coif3',
 'coif4',
 'coif5',
 'coif6',
 'coif7',
 'coif8',
 'coif9',
 'coif10',
 'coif11',
 'coif12',
 'coif13',
 'coif14',
 'coif15',
 'coif16',
 'coif17',
 'db1',
 'db2',
 'db3',
 'db4',
 'db5',
 'db6',
 'db7',
 'db8',
 'db9',
 'db10',
 'db11',
 'db12',
 'db13',
 'db14',
 'db15',
 'db16',
 'db17',
 'db18',
 'db19',
 'db20',
 'db21',
 'db22',
 'db23',
 'db24',
 'db25',
 'db26',
 'db27',
 'db28',
 'db29',
 'db30',
 'db31',
 'db32',
 'db33',
 'db34',
 'db35',
 'db36',
 'db37',
 'db38',
 'dmey',
 'fbsp',
 'gaus1',
 'gaus2',
 'gaus3',
 'gaus4',
 'gaus5',
 'gaus6',
 'gaus7',
 'gaus8',
 'haar',
 'mexh',
 'morl',
 'rbio1.1',
 'rbio1.3',
 'rbio1.5',
 'rbio2.2',
 'rbio2.4',
 'rbio2.6',
 'rbio2.8',
 'rbio3.1',

In [101]:
# Builtin Daubechies wavelet
pywt.wavelist('db')

['db1',
 'db2',
 'db3',
 'db4',
 'db5',
 'db6',
 'db7',
 'db8',
 'db9',
 'db10',
 'db11',
 'db12',
 'db13',
 'db14',
 'db15',
 'db16',
 'db17',
 'db18',
 'db19',
 'db20',
 'db21',
 'db22',
 'db23',
 'db24',
 'db25',
 'db26',
 'db27',
 'db28',
 'db29',
 'db30',
 'db31',
 'db32',
 'db33',
 'db34',
 'db35',
 'db36',
 'db37',
 'db38']

(1323648,)

In [193]:
# Single level Discrete Wavelet Transform. Parameters:
# data : array_like - Input signal
# wavelet : Wavelet object or name - Wavelet to use
# mode : str, optional - Signal extension mode, see Modes.
# axis: int, optional - Axis over which to compute the DWT. If not given, the last axis is used.

# Returns: (cA, cD) : tuple - Approximation and detail coefficients.
(cA, cD) = pywt.dwt(y, 'db4')
cA.shape, cA, cD.shape, cD

((661827,),
 array([0., 0., 0., ..., 0., 0., 0.], dtype=float32),
 (661827,),
 array([0., 0., 0., ..., 0., 0., 0.], dtype=float32))

In [116]:
# Compute the maximum useful level of decomposition. Parameters:
# data_len : int - Input data length.
# filter_len : int, str or Wavelet - The wavelet filter length. Alternatively, the name of a discrete wavelet or a Wavelet object can be specified.

# Returns: max_level : int - Maximum level.
pywt.dwt_max_level(len(y), 'db4')

17

In [194]:
# Compute the maximum level of decomposition for n-dimensional data.
# This returns the maximum number of levels of decomposition suitable for use with wavedec, wavedec2 or wavedecn.
# Parameters:
# shape : sequence of ints - Input data shape.
# wavelet : Wavelet object or name string, or tuple of wavelets - Wavelet to use. This can also be a tuple containing a wavelet to apply along each axis in axes.
# axes : sequence of ints, optional - Axes over which to compute the DWT. Axes may not be repeated.

# Returns: - level : int - Maximum level.
level = pywt.dwtn_max_level(y.shape, 'db4')
level

17

In [195]:
# Multilevel 1D Discrete Wavelet Transform of data. Parameters:
# data: array_like - Input data
# wavelet : Wavelet object or name string - Wavelet to use
# mode : str, optional - Signal extension mode, see Modes.
# level : int, optional - Decomposition level (must be >= 0). If level is None (default) then it will be calculated
# using the dwt_max_level function.
# axis: int, optional - Axis over which to compute the DWT. If not given, the last axis is used.

# Returns:[cA_n, cD_n, cD_n-1, …, cD2, cD1] : list
# Ordered list of coefficients arrays where n denotes the level of decomposition. 
# The first element (cA_n) of the result is approximation coefficients array and the following elements
# (cD_n - cD_1) are details coefficients arrays.
db_coeffs = pywt.wavedec(y, 'db4', level=2)
cA2, cD2, cD1 = db_coeffs
cA2.shape, cA2, cD2.shape, cD2, cD1.shape, cD1

((330917,),
 array([0., 0., 0., ..., 0., 0., 0.], dtype=float32),
 (330917,),
 array([0., 0., 0., ..., 0., 0., 0.], dtype=float32),
 (661827,),
 array([0., 0., 0., ..., 0., 0., 0.], dtype=float32))

In [157]:
np.var(cA2)

0.06052318